In [10]:
import numpy as np
import pandas as pd
import matplotlib as plt
import os
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, load_img

Using TensorFlow backend.


In [9]:
os.listdir('/home/ipc-admin/Documents/CHEST/chest_xray/chest_xray')

['val',
 'train',
 'PNEMUNIA Detection in XRAY using CNN.ipynb',
 '.ipynb_checkpoints',
 'test',
 '.DS_Store']

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255) 

training_set = train_datagen.flow_from_directory('/home/ipc-admin/Documents/CHEST/chest_xray/chest_xray/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory('/home/ipc-admin/Documents/CHEST/chest_xray/chest_xray/val/',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary')

test_set = test_datagen.flow_from_directory('/home/ipc-admin/Documents/CHEST/chest_xray/chest_xray/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')



Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [12]:
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(32, (3, 3), activation="relu"))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Flatten())
cnn.add(Dense(activation = 'relu', units = 128))
cnn.add(Dense(activation = 'sigmoid', units = 1))
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total para

In [14]:
cnn_model = cnn.fit_generator(training_set,
                         steps_per_epoch = 150,
                         epochs = 10,
                         validation_data = validation_generator,
                         validation_steps = 500) 

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
150/150 [==============================] - 105s 699ms/step - loss: 0.3772 - acc: 0.8317 - val_loss: 0.7981 - val_acc: 0.6250
Epoch 2/10
150/150 [==============================] - 105s 700ms/step - loss: 0.2338 - acc: 0.9046 - val_loss: 0.3538 - val_acc: 0.8125
Epoch 3/10
150/150 [==============================] - 107s 711ms/step - loss: 0.2249 - acc: 0.9087 - val_loss: 0.2106 - val_acc: 0.9375
Epoch 4/10
150/150 [==============================] - 105s 699ms/step - loss: 0.2068 - acc: 0.9142 - val_loss: 0.3986 - val_acc: 0.6875
Epoch 5/10
150/150 [==============================] - 102s 683ms/step - loss: 0.1792 - acc: 0.9300 - val_loss: 0.4358 - val_acc: 0.6875
Epoch 6/10
150/150 [==============================] - 109s 725ms/step - loss: 0.1816 - acc: 0.9285 - val_loss: 0.4852 - val_acc: 0.6875
Epoch 7/10
150/150 [==============================] - 105s 697ms/step - loss: 0.1756 - acc: 0.9313 - val_loss: 0.3854 - val_acc: 0.8125


In [15]:
test_accu = cnn.evaluate_generator(test_set,steps=500)

print('The testing accuracy is :',test_accu[1]*100, '%')



The testing accuracy is : 89.26282051282051 %


In [20]:
cnn.save_weights("chestmodel.h5")

In [16]:
from keras.preprocessing import image
from PIL import Image
test_image = image.load_img('/home/ipc-admin/Documents/CHEST/chest_xray/chest_xray/test/PNEUMONIA/person89_bacteria_440.jpeg')
test_image = test_image.resize((64, 64), Image.ANTIALIAS)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)

In [17]:
value = cnn.predict_classes(test_image)

In [18]:
if value[0][0]==1:
    print("PNEUMONIA")
else:
    print("NORMAL")

PNEUMONIA
